In [13]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 
import os
import pandas as pd
import skimage.io as io 
import tensorflow as tf


from keras.applications import ResNet50 
from keras.applications.resnet import preprocess_input
from keras.layers import (Dense,  
                          Flatten)
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import (accuracy_score,
                             confusion_matrix,
                             precision_score,
                             recall_score,
                             roc_auc_score)
from tensorflow import keras

In [30]:
my_dir = 'C:\\Users\\ianae\\Mariano_Machine_Learning\\Tarea5\\'

In [15]:
def resize(dir):
    for p in os.listdir(dir):
        if p[0] == '0':
            img = plt.imread(dir+'/'+p)
            img = cv2.resize(img, (250, 250))
            io.imsave(dir+'/'+p, img)

In [16]:
resize('.\\test\\PNEUMONIA')
resize('.\\test\\NORMAL')

In [32]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   brightness_range  = (0.5, 1.5))
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_dir = ''.join([my_dir, 'train'])
val_dir = ''.join([my_dir, 'val'])
test_dir = ''.join([my_dir, 'test'])

train_generator = train_datagen.flow_from_directory(
                        train_dir,              
                        target_size= (250, 250), 
                        batch_size = 20,   
                        shuffle    = True,
                        class_mode = 'binary')    

val_generator = val_datagen.flow_from_directory(
                        val_dir,
                        target_size=(250, 250),
                        batch_size=20,
                        class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [36]:
train_generator

In [18]:
#Cargo la red convulocional de resnet
resnet = ResNet50(include_top=False, 
                   weights='imagenet', 
                   input_shape=(250,250,3))
#Congelo los parametros
#no son entrenables 
resnet.trainable = False

model = Sequential()
model.add(resnet)   # modelo base agradado como una capa!
#Pongo todo en un vector para ajustarlo a la capa densa
model.add(Flatten())
#Creo la capa densa 
model.add(Dense(256, activation='relu'))
#model.add(layers.Dropout(0.3))  # a ver
model.add(Dense(1, activation='sigmoid'))

In [19]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=1e-6),
              metrics=['acc'])

In [23]:
type(train_generator)

keras.preprocessing.image.DirectoryIterator

In [29]:
history = model.fit(train_generator,
                    steps_per_epoch = len(train_generator)//30,
                    epochs = 30,
                    validation_data = val_generator,
                    validation_steps= 50,
                    verbose         = 1)

Epoch 1/30
8/8 [==============================] - 21s 3s/step - loss: 0.0295 - acc: 0.9937 - val_loss: 0.1038 - val_acc: 1.0000
Epoch 2/30
2/8 [======>.......................] - ETA: 11s - loss: 0.0505 - acc: 0.9750

KeyboardInterrupt: 

In [26]:
acc      = history.history['acc']
val_acc  = history.history['val_acc']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Entrenamiento acc')
plt.plot(epochs, val_acc, 'b', label='Validación acc')
plt.title('Accuracy - exactitud de entrenamiento y validación')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Entrenamiento loss')
plt.plot(epochs, val_loss, 'b', label='Validación loss')
plt.title('Loss - función objetivo en entrenamiento y prueba')
plt.legend()

plt.show()

KeyError: 'val_acc'

In [ ]:
model.save(dir + 'model1')

In [ ]:
model_path = dir + 'model1'

In [ ]:
model = keras.models.load_model(model_path)

In [ ]:
resnet.trainable = True

resnet_layers = resnet.layers[:-8]

set_trainable = False
for layer in resnet.layers:
    if layer.name in resnet_layers:
        layer.trainable = True
    else:
        layer.trainable = False 
        

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=1e-5),
              metrics=['acc'])

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch = 50,
                    epochs = 30,
                    validation_data = val_generator,
                    validation_steps= 50,
                    verbose         = 1)

In [ ]:
def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

acc      = history.history['acc']
val_acc  = history.history['val_acc']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
alpha = 0.9

plt.plot(epochs, smooth_curve(acc,     alpha), 'bo', label='Entrenamiento acc')
plt.plot(epochs, smooth_curve(val_acc, alpha), 'b', label='Validación acc')
plt.title('Accuracy - exactitud de entrenamiento y validación')
plt.legend()

plt.figure()

plt.plot(epochs, smooth_curve(loss,    alpha), 'bo', label='Entrenamiento loss')
plt.plot(epochs, smooth_curve(val_loss,alpha), 'b', label='Validación loss')
plt.title('Loss - función objetivo en entrenamiento y prueba')
plt.legend()

plt.show()

In [ ]:
model.save(dir + 'model2')

In [ ]:
model_path = dir + 'model2'

In [ ]:
model = keras.models.load_model(model_path)

In [ ]:
resnet.trainable = True

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=1e-5),
              metrics=['acc'])

In [ ]:
checkpoint_filepath = 'C:\\Users\\ianae\\Mariano_Machine_Learning\\Tarea5\\checkpoint3'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only = True,
    monitor = 'val_accuracy',
    mode = 'max',
    save_best_only=True
)

# Model weights are saved at the end of every epoch, if it's the best seen
# so far.
history = model.fit(train_generator,
                    steps_per_epoch = 50,
                    epochs = 30,
                    callbacks = [model_checkpoint_callback],
                    validation_data = val_generator,
                    validation_steps= 50,
                    verbose         = 1)


# The model weights (that are considered the best) are loaded into the
# model.
model.load_weights(checkpoint_filepath)

In [ ]:
acc      = history.history['acc']
val_acc  = history.history['val_acc']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
alpha = 0.9

plt.plot(epochs, smooth_curve(acc,     alpha), 'bo', label='Entrenamiento acc')
plt.plot(epochs, smooth_curve(val_acc, alpha), 'b', label='Validación acc')
plt.title('Accuracy - exactitud de entrenamiento y validación')
plt.legend()

plt.figure()

plt.plot(epochs, smooth_curve(loss,    alpha), 'bo', label='Entrenamiento loss')
plt.plot(epochs, smooth_curve(val_loss,alpha), 'b', label='Validación loss')
plt.title('Loss - función objetivo en entrenamiento y prueba')
plt.legend()

plt.show()

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(
                        test_dir,              
                        target_size= (250, 250), 
                        batch_size = 20,   
                        shuffle    = True,
                        class_mode = 'binary')    

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

# Predicción en el conjunto de test
y_pred = model.predict(test_generator, verbose=1)
y_true = test_generator.classes